In [1]:
# encoding=utf-8

import jieba
import sys
import re
import time
import string
from sklearn import feature_extraction
from sklearn.pipeline import Pipeline,FeatureUnion

from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import CountVectorizer

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_selection import SelectKBest, chi2,mutual_info_classif,f_classif

import pickle

import numpy as np
from sklearn import cross_validation
from sklearn.cross_validation import KFold  
from sklearn.linear_model import LogisticRegression  
from sklearn.naive_bayes import GaussianNB  
from sklearn.neighbors import KNeighborsClassifier   
from sklearn import svm  
from sklearn.tree import DecisionTreeClassifier  
from sklearn.ensemble import RandomForestClassifier  

from sklearn.datasets.base import Bunch
from sklearn.linear_model import SGDClassifier
from sklearn.metrics import confusion_matrix, f1_score
from sklearn.ensemble import ExtraTreesClassifier

import xgboost as xgb
from xgboost import XGBClassifier
from sklearn import metrics

from sklearn.base import BaseEstimator, TransformerMixin
from collections import defaultdict

D:\software\conda\lib\site-packages\sklearn\cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [2]:
corpus = []
lab = []

f = open("i_p_1.txt","r+", encoding='UTF-8')
for content in f:
    corpus.append(content.split("\t")[0])
    lab.append(1)
f.close()

#neg sample
f = open("i_n_1.txt","r+", encoding='UTF-8')
for content in f:
    corpus.append(content.split("\t")[0])
    lab.append(0)
f.close()


FileNotFoundError: [Errno 2] No such file or directory: 'i_p_1.txt'

In [3]:
class StatsFeatures(BaseEstimator, TransformerMixin):
    
    def __init__(self):
        self.neg = set()
        f = open("neg_words.txt","r+", encoding='UTF-8')
        for content in f:
            self.neg.add(content)
        f.close()

    def fit(self, X, y=None):
        return self

    def getcnt(self,x):
        return len(list(set(x)))

    def getnegcnt(self,x):
        negcnt = 0
        words = x.split()
        for w in words:
            if w in self.neg:
                negcnt = negcnt+1
        return negcnt
    
    def transform(self, X):
        return [[len(x),self.getcnt(x),self.getcnt(x)/len(x),self.getnegcnt(x),self.getnegcnt(x)/len(x)] for x in X]
    

In [ ]:
vectorizer = TfidfVectorizer()

ch2 = SelectKBest(chi2, k=40000)
stats = StatsFeatures()
lab = np.asarray(lab)

combined_features = FeatureUnion([
    (Pipeline([
        ("tf_idf", vectorizer), 
        ("ch2", ch2)
    ])),
    ("stats",stats)
])

params = {'nthread':8,'max_depth':6, 'eta':0.2, 'eval_metric':'merror', 'silent':1, 'objective':'multi:softmax', 'num_class':2}  # 参数

weight = combined_features.fit(corpus, y=lab).transform(corpus)

In [4]:
pipeline = Pipeline([
    ('features', FeatureUnion([
        ('tf_idf', Pipeline([
            ('counts', CountVectorizer()),
            ('tf_idf', TfidfTransformer()),
            ('chi', SelectKBest(chi2, k=20000))
        ])),
        ('len_stats', StatsFeatures())
    ])),
    ('classifier', XGBClassifier(max_depth=7,objective='multi:softmax', num_class=2))
])

#params = {'nthread':8,'max_depth':6, 'eta':0.2, 'eval_metric':'merror', 'silent':1, 'objective':'multi:softmax', 'num_class':2}  # 参数
#
pipeline.fit(corpus, lab)
 
#kf = KFold(len(corpus), n_folds=10, shuffle=True)    
#result_set = [(pipeline.fit(corpus[train], lab[train]).predict(corpus[test]), test) for train, test in kf]    
#score = [accuracy(lab[result[1]], result[0]) for result in result_set]    
#print(score,np.average(score)) 

Pipeline(memory=None,
     steps=[('features', FeatureUnion(n_jobs=1,
       transformer_list=[('tf_idf', Pipeline(memory=None,
     steps=[('counts', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0...tate=0, reg_alpha=0, reg_lambda=1, scale_pos_weight=1,
       seed=None, silent=True, subsample=1))])

In [5]:
pipeline.score(corpus, lab)  

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


0.9953455899464743

In [7]:
corpus_test = []
f = open("i_test_1.txt","r+", encoding='UTF-8')
for content in f:
    if(len(content)>1):
        corpus_test.append(content.strip())
f.close()

res = pipeline.predict(corpus_test)
print( 1-float(sum(res))/len(res),len(res))

0.91 200


C:\ProgramData\Anaconda3\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


In [6]:
import joblib
joblib.dump(pipeline, "NP_model_i.pkl.z")  

['model_b.pkl.z']